In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sympy.physics.units as u
import sympy as sp
import numpy as np
import logging as log
import functools
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
pd.set_option('display.max_columns', None)
%matplotlib inline
from IPython.display import display, HTML

import sys, os, importlib
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import miptlabs as ml
importlib.reload(ml)

importlib.reload(log)
log.basicConfig(level=log.INFO, format='LOG %(message)s', stream=sys.stdout)

sp.Basic.__str__=lambda expr, **settings:ml.PQStrPrinter(settings).doprint(expr)

sp.__version__

'1.1.2.dev'

In [3]:
data = pd.read_csv('5.1.csv')
display(data)

,#,X3,d,X4,Lambda_2_N,n,N,Lambda_2_N.1,n.1,N.1
0,1,130,4.5,115,15.0,3,5.0,8.5,1,1
1,2,120,4.0,105,15.0,5,NaN,9.0,1,1
2,3,110,4.0,95,13.5,5,NaN,8.5,1,1
3,4,100,3.0,85,12.0,5,NaN,8.0,1,1


In [4]:
# Погрешность измерения длины одинакова: по 5 мм с каждой стороны
dist_sigma = np.sqrt(2)*0.5*u.mm
coord_sigma = 1*u.mm

df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df1['X3'] = ml.get_column(data, 'X3', lambda row: ml.PQ(row['X3']*u.cm, sigma=coord_sigma))
df1['d'] = ml.get_column(data, 'd', lambda row: ml.PQ(row['d']*u.mm, sigma=dist_sigma))
df2['X4'] = ml.get_column(data, 'X4', lambda row: ml.PQ(row['X4']*u.cm, sigma=coord_sigma))
df2['Lambda_2_N'] = ml.get_column(data, 'Lambda_2_N', lambda row: ml.PQ(row['Lambda_2_N']*u.mm, sigma=dist_sigma))
df2['n'] = data['n']
df2['N'] = 2*data['n']-1
df2['Lambda_2'] = df2['Lambda_2_N']/df2['N']
df3['X4'] = df2['X4']
df3['Lambda_2_N'] = ml.get_column(data, 'Lambda_2_N.1', lambda row: ml.PQ(row['Lambda_2_N.1']*u.mm, sigma=dist_sigma))
df3['n'] = data['n.1']
df3['N'] = 2*data['n.1']-1
df3['Lambda_2'] = df3['Lambda_2_N']/df3['N']

display(df1)
display(df2)
display(df3)


,X3,d
0,130.00±0.10 cm (0.077%),4.5±0.7 mm (16%)
1,120.00±0.10 cm (0.083%),4.0±0.7 mm (18%)
2,110.00±0.10 cm (0.091%),4.0±0.7 mm (18%)
3,100.00±0.10 cm (0.10%),3.0±0.7 mm (24%)


,X4,Lambda_2_N,n,N,Lambda_2
0,115.00±0.10 cm (0.087%),15.0±0.7 mm (4.7%),3,5,3.00±0.14 mm (4.7%)
1,105.00±0.10 cm (0.095%),15.0±0.7 mm (4.7%),5,9,1.67±0.08 mm (4.7%)
2,95.00±0.10 cm (0.11%),13.5±0.7 mm (5.2%),5,9,1.50±0.08 mm (5.2%)
3,85.00±0.10 cm (0.12%),12.0±0.7 mm (5.9%),5,9,1.33±0.08 mm (5.9%)


,X4,Lambda_2_N,n,N,Lambda_2
0,115.00±0.10 cm (0.087%),8.5±0.7 mm (8.3%),1,1,8.5±0.7 mm (8.3%)
1,105.00±0.10 cm (0.095%),9.0±0.7 mm (7.9%),1,1,9.0±0.7 mm (7.9%)
2,95.00±0.10 cm (0.11%),8.5±0.7 mm (8.3%),1,1,8.5±0.7 mm (8.3%)
3,85.00±0.10 cm (0.12%),8.0±0.7 mm (8.8%),1,1,8.0±0.7 mm (8.8%)


In [5]:
X1 = ml.PQ(140*u.cm, sigma=1*u.mm) # Лазер
X2 = ml.PQ(15*u.cm, sigma=1*u.mm) # Экран
lambd = ml.PQ(661*10**-9*u.m, is_const=True) # Длина волны лазера
# X3 - кювета
# X4 - линза
F = ml.PQ(36*u.mm, is_const=True) # Фокусное расстояние линзы



In [6]:
# Пункт 3
df1['L'] = df1['X3']-X2
df1['alpha'] = ml.pqarray(df1['d']/df1['L']).repr_as(1)

In [7]:
# Пункты 5-9
df2['f'] = df2['X4']-X2
df2['d'] = (df2['f']*F)/(df2['f']-F)
df2['Gamma'] = df2['f']/df2['d']

df2['Lambda_1'] = df2['Lambda_2']/df2['Gamma']
df2['alpha'] = ml.pqarray(lambd/df2['Lambda_1']).repr_as(1)

In [27]:
# Пункт 12
df3['f'] = df3['X4']-X2
df3['d'] = (df3['f']*F)/(df3['f']-F)
df3['Gamma'] = df3['f']/df3['d']

df3['Lambda_1'] = df3['Lambda_2']/df3['Gamma']
df3['alpha'] = ml.pqarray(lambd/df3['Lambda_1']).repr_as(1)

eta_water = 4/3
k = (df2['alpha']/df3['alpha'])
df3['eta_glass'] = (eta_water*k-1)/(k-1)
eta_glass_mean = ml.get_mean(df3['eta_glass'].loc[1:])
print(eta_glass_mean) # Лёгкий крон (ЛК3)
eta_glass_true = 1.4874

beta1plus2 = df2['alpha']/(eta_glass_true-1) #df3['eta_glass']
df3['beta_mean'] = beta1plus2/2
beta_mean_mean = ml.get_mean(df3['beta_mean'])
print(beta_mean_mean)

1.40±0.22  (16%)
(8.7±0.9)*10^-3  (11%)


In [28]:
display(df1)
display(df2)
display(df3)

,X3,d,L,alpha
0,130.00±0.10 cm (0.077%),4.5±0.7 mm (16%),115.00±0.14 cm (0.12%),(3.9±0.6)*10^-3 (16%)
1,120.00±0.10 cm (0.083%),4.0±0.7 mm (18%),105.00±0.14 cm (0.13%),(3.8±0.7)*10^-3 (18%)
2,110.00±0.10 cm (0.091%),4.0±0.7 mm (18%),95.00±0.14 cm (0.15%),(4.2±0.7)*10^-3 (18%)
3,100.00±0.10 cm (0.10%),3.0±0.7 mm (24%),85.00±0.14 cm (0.17%),(3.5±0.8)*10^-3 (24%)


,X4,Lambda_2_N,n,N,Lambda_2,f,d,Gamma,Lambda_1,alpha
0,115.00±0.10 cm (0.087%),15.0±0.7 mm (4.7%),3,5,3.00±0.14 mm (4.7%),100.00±0.14 cm (0.14%),3.734±0.008 cm (0.20%),26.78±0.07 (0.25%),0.112±0.005 mm (4.7%),(59.00±2.80)*10^-4 (4.7%)
1,105.00±0.10 cm (0.095%),15.0±0.7 mm (4.7%),5,9,1.67±0.08 mm (4.7%),90.00±0.14 cm (0.16%),3.750±0.009 cm (0.23%),24.00±0.07 (0.28%),0.069±0.003 mm (4.7%),(9.5±0.4)*10^-3 (4.7%)
2,95.00±0.10 cm (0.11%),13.5±0.7 mm (5.2%),5,9,1.50±0.08 mm (5.2%),80.00±0.14 cm (0.18%),3.770±0.010 cm (0.26%),21.22±0.07 (0.31%),0.071±0.004 mm (5.2%),(9.4±0.5)*10^-3 (5.2%)
3,85.00±0.10 cm (0.12%),12.0±0.7 mm (5.9%),5,9,1.33±0.08 mm (5.9%),70.00±0.14 cm (0.20%),3.795±0.011 cm (0.29%),18.44±0.07 (0.36%),0.072±0.004 mm (5.9%),(9.1±0.5)*10^-3 (5.9%)


,X4,Lambda_2_N,n,N,Lambda_2,f,d,Gamma,Lambda_1,alpha,eta_glass,beta_mean
0,115.00±0.10 cm (0.087%),8.5±0.7 mm (8.3%),1,1,8.5±0.7 mm (8.3%),100.00±0.14 cm (0.14%),3.734±0.008 cm (0.20%),26.78±0.07 (0.25%),0.317±0.026 mm (8.3%),(20.80±1.70)*10^-4 (8.3%),1.52±0.30 (20%),(60.50±2.90)*10^-4 (4.7%)
1,105.00±0.10 cm (0.095%),9.0±0.7 mm (7.9%),1,1,9.0±0.7 mm (7.9%),90.00±0.14 cm (0.16%),3.750±0.009 cm (0.23%),24.00±0.07 (0.28%),0.375±0.029 mm (7.9%),(17.60±1.40)*10^-4 (7.9%),1.41±0.22 (15%),(9.8±0.5)*10^-3 (4.7%)
2,95.00±0.10 cm (0.11%),8.5±0.7 mm (8.3%),1,1,8.5±0.7 mm (8.3%),80.00±0.14 cm (0.18%),3.770±0.010 cm (0.26%),21.22±0.07 (0.31%),0.40±0.03 mm (8.3%),(16.50±1.40)*10^-4 (8.3%),1.40±0.23 (16%),(9.6±0.5)*10^-3 (5.2%)
3,85.00±0.10 cm (0.12%),8.0±0.7 mm (8.8%),1,1,8.0±0.7 mm (8.8%),70.00±0.14 cm (0.20%),3.795±0.011 cm (0.29%),18.44±0.07 (0.36%),0.43±0.04 mm (8.8%),(15.20±1.30)*10^-4 (8.8%),1.40±0.25 (18%),(9.4±0.6)*10^-3 (5.9%)


In [41]:
columns = []
for column in df1.columns:
    columns.append( column.replace('_', ''))
df1.columns = columns

columns = []
for column in df2.columns:
    columns.append( column.replace('_', ''))
df2.columns = columns

columns = []
for column in df3.columns:
    columns.append( column.replace('_', ''))
df3.columns = columns

table_to_print1 = ml.prepare_to_print(df1)
table_to_print2 = ml.prepare_to_print(df2).iloc[:,:15]
table_to_print3 = ml.prepare_to_print(df2).iloc[:,15:]
table_to_print4 = ml.prepare_to_print(df3).iloc[:,:15]
table_to_print5 = ml.prepare_to_print(df3).iloc[:,15:]
ml.write_latex('table.tex', [table_to_print1, table_to_print2, table_to_print3, table_to_print4, table_to_print5], horizontal=True)
